In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
with open ('phones.txt', 'r') as f:
    phones = f.readlines()
    
original_phones = [phone.strip() for phone in phones]

phones = [phones[i].strip().lower() for i in range(len(phones))]

url_part = ["+".join(phones[i].split()) for i in range(len(phones))]

In [3]:
reviews_dict = {"Phone Name" : [], "Source" : "Digital Trends", "URL" : [], "Review" : []}

count = 0
for i in range(len(phones)):
    try:
        addr = f"https://www.digitaltrends.com/?s={url_part[i]}+review"
        digital_trends = requests.get(addr)
        dt_soup = BeautifulSoup(digital_trends.content, "html5lib")
        search_res = dt_soup.find_all('div', attrs={"class" : "b-snippet b-grid__cols2 b-grid__rows2"})
        a_tag = search_res[0].find("a")
        review_link = a_tag.get('href')
        phone_review = requests.get(review_link)
        pr_soup = BeautifulSoup(phone_review.content, "html5lib")
        review = pr_soup.find('article', attrs={"id" : "dt-post-content"})
        review_paras = review.find_all('p')
        final_review = ""
        for para in review_paras:
            final_review += para.text
        final_review = final_review.strip().replace("\n", "")
        reviews_dict["Phone Name"].append(original_phones[i])
        reviews_dict["Review"].append(final_review)
        reviews_dict["URL"].append(review_link)
    except:
        count += 1

review_df = pd.DataFrame(reviews_dict)

In [4]:
review_df

,Phone Name,Source,URL,Review
0,IPhone 13,Digital Trends,https://www.digitaltrends.com/cell-phone-revie...,The iPhone 13 is the new iPhone that Apple exp...
1,IPhone 13 Pro,Digital Trends,https://www.digitaltrends.com/cell-phone-revie...,I’ll say it upfront; the iPhone 13 Pro is the ...
2,Moto G (2021),Digital Trends,https://www.digitaltrends.com/cell-phone-revie...,The Moto G series has long offered excellent v...
3,Moto G10,Digital Trends,https://www.digitaltrends.com/mobile/top-tech-...,"Since September of last year, OnePlus’ CEO Car..."
4,Motorola Edge (2021),Digital Trends,https://www.digitaltrends.com/mobile/motorola-...,The Motorola Edge 5G UW (2021) is a Verizon-on...
...,...,...,...,...
119,Xiaomi Mi 8,Digital Trends,https://www.digitaltrends.com/cell-phone-revie...,What better way for Xiaomi to announce its ent...
120,Xiaomi Mi A2,Digital Trends,https://www.digitaltrends.com/cell-phone-revie...,There was a time when many would not have paid...
121,Xiaomi Mi MIX 2S,Digital Trends,https://www.digitaltrends.com/cell-phone-revie...,As prices for flagship smartphones have climbe...
122,Xiaomi Mi MIX 3,Digital Trends,https://www.digitaltrends.com/cell-phone-revie...,The Xiaomi Mi Mix 3 is a highly-capable phone ...


In [5]:
review_df.to_csv("CSVs/digital_trends.csv")